# How to set a prior given T = 2 non-zero effects?

Assume two non-zero effects $T = 2$. We set PVE $ = 0.05$ so that SuSiE has certain power around $0.5$. 
We also set PVE $ = 0.7$ so that SuSiE has power around $0.9$. We investigate SuSiE performance under different priors.

## Results

** - Summary: prior has a modest influence on SuSiE's performance. The size of confidence sets in the randomly generated X data is always 1. When PVE is large, the number of confidence sets produced is close to true number of non-zero effects.**


In [6]:
dscout.summary[dscout.summary$pve==0.05,]

,effect_num,pve,prior,power,fdr,cs_size,cs_num,top_hit_rate,avg_purity
20,2,0.05,0.01,0.50,0,1,1.063830,1,1
86,2,0.05,0.02,0.51,0,1,1.085106,1,1
152,2,0.05,0.03,0.52,0,1,1.083333,1,1
218,2,0.05,0.05,0.52,0,1,1.083333,1,1
284,2,0.05,0.10,0.52,0,1,1.083333,1,1
350,2,0.05,0.20,0.52,0,1,1.083333,1,1
416,2,0.05,0.40,0.52,0,1,1.083333,1,1
482,2,0.05,0.50,0.52,0,1,1.083333,1,1
548,2,0.05,0.70,0.52,0,1,1.083333,1,1
614,2,0.05,0.90,0.52,0,1,1.083333,1,1


In [7]:
dscout.summary[dscout.summary$pve==0.7,]

,effect_num,pve,prior,power,fdr,cs_size,cs_num,top_hit_rate,avg_purity
50,2,0.7,0.01,0.9,0,1,1.8,1,1
116,2,0.7,0.02,0.9,0,1,1.8,1,1
182,2,0.7,0.03,0.9,0,1,1.8,1,1
248,2,0.7,0.05,0.9,0,1,1.8,1,1
314,2,0.7,0.10,0.9,0,1,1.8,1,1
380,2,0.7,0.20,0.9,0,1,1.8,1,1
446,2,0.7,0.40,0.9,0,1,1.8,1,1
512,2,0.7,0.50,0.9,0,1,1.8,1,1
578,2,0.7,0.70,0.9,0,1,1.8,1,1
644,2,0.7,0.90,0.9,0,1,1.8,1,1


## Code details

In [1]:
dscout_Q2 = readRDS('random_Q2.rds')
dscout_Q2 = dscout_Q2[!is.na(dscout_Q2$sim_gaussian.output.file),]
dscout_Q2 = dscout_Q2[!is.na(dscout_Q2$susie_prior.output.file),]

In [2]:
dscout_df = data.frame(dscout_Q2$sim_gaussian.effect_num, dscout_Q2$sim_gaussian.pve, dscout_Q2$susie_prior.prior,
                       dscout_Q2$score.hit, dscout_Q2$score.signal_num, dscout_Q2$score.cs_medianSize,
                       dscout_Q2$score.top_hit, dscout_Q2$sim_gaussian.mean_corX, dscout_Q2$susie_prior.avg_purity)
names(dscout_df) = c('effect_num', 'pve', 'prior','hit', 'cs_num', 'cs_size', 'top_hit', 'corX', 'avg_purity')

In [3]:
hitsum.summary = aggregate(hit ~ effect_num + pve + prior, dscout_df, sum)
hitmean.summary = aggregate(hit ~ effect_num + pve + prior, dscout_df, mean)
power.summary = hitmean.summary
power.summary$power = power.summary$hit / power.summary$effect_num
fdr.summary = aggregate(cs_num ~ effect_num + pve + prior, dscout_df, sum)
fdr.summary$fdr = round(1 - hitsum.summary$hit / fdr.summary$cs_num, 4)
meannonzero = function(x){mean(x[x!=0])}
cs_num.summary = aggregate(cs_num ~ effect_num + pve + prior, dscout_df, meannonzero)
setsize.summary = aggregate(cs_size ~ effect_num + pve + prior, dscout_df, meannonzero)
tophit.summary = aggregate(top_hit ~ effect_num + pve + prior, dscout_df, sum)
tophit.summary$tophit_rate = round(tophit.summary$top_hit / fdr.summary$cs_num , 4)
dscout_df$avg_purity[is.na(dscout_df$avg_purity)]=0
purity.summary = aggregate(avg_purity ~ effect_num + pve + prior, dscout_df, meannonzero)

In [4]:
dscout.summary = data.frame(power.summary$effect_num, power.summary$pve, power.summary$prior,
                            power.summary$power, fdr.summary$fdr, 
                            setsize.summary$cs_size, cs_num.summary$cs_num, 
                            tophit.summary$tophit_rate, purity.summary$avg_purity)
names(dscout.summary) = c('effect_num', 'pve', 'prior','power', 'fdr', 'cs_size', 'cs_num','top_hit_rate', 'avg_purity')

In [5]:
dscout.summary = dscout.summary[dscout.summary$effect_num==2, ]